In [13]:
import pandas as pd
pd.set_option('display.max_columns', 200)
df = pd.read_csv("freight_ops_dataset.csv")
print(df.shape)
display(df.head())
display(df.dtypes)
display(df.isnull().sum())
display(df.describe(include='all').transpose())


(500, 10)


,shipment_id,region,route,distance_miles,expected_days,delivery_days,cost_usd,expected_date,delivery_date,delayed
0,d0ce1724-3ebd-4919-94c8-130e450f1bdc,West,A-B,927.47,3,3,2504.17,2025-07-27,2025-07-27,False
1,86a7a677-e5e4-40db-b1a1-d73463be5746,West,E-F,1415.22,2,2,3354.07,2025-07-31,2025-07-31,False
2,d1124997-9909-44ef-8270-c7013d9a3229,South,D-E,1424.20,5,8,4685.62,2025-07-22,2025-07-25,True
3,dba33228-a302-4190-bef6-2cce61db1100,Northeast,B-C,1989.21,2,5,3680.04,2025-07-02,2025-07-05,True
4,2f706978-6205-424f-a861-3032f945ca6f,Northeast,D-E,810.41,3,5,1393.91,2025-07-19,2025-07-21,True


shipment_id        object
region             object
route              object
distance_miles    float64
expected_days       int64
delivery_days       int64
cost_usd          float64
expected_date      object
delivery_date      object
delayed              bool
dtype: object

shipment_id       0
region            0
route             0
distance_miles    0
expected_days     0
delivery_days     0
cost_usd          0
expected_date     0
delivery_date     0
delayed           0
dtype: int64

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
shipment_id,500,500,439f383d-8c79-45c4-81fb-f02b4e1a6686,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,500,4,Northeast,141,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route,500,5,A-B,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN
distance_miles,500.0,NaN,NaN,NaN,1100.82264,539.288321,110.76,628.725,1104.3,1593.8625,1998.54
expected_days,500.0,NaN,NaN,NaN,4.462,1.760328,2.0,3.0,4.0,6.0,7.0
delivery_days,500.0,NaN,NaN,NaN,5.52,2.311772,1.0,4.0,6.0,7.0,10.0
cost_usd,500.0,NaN,NaN,NaN,2770.84308,1492.21348,199.37,1472.1675,2664.705,3835.6725,6708.66
expected_date,500,34,2025-07-26,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delivery_date,500,30,2025-07-23,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delayed,500,2,True,305,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df['expected_date'] = pd.to_datetime(df['expected_date'], errors='coerce')
df['delivery_date'] = pd.to_datetime(df['delivery_date'], errors='coerce')
# if dataset has numeric expected/delivery days, keep them too:
if 'delivery_days' in df.columns and 'expected_days' in df.columns:
    df['delay_days_by_value'] = df['delivery_days'] - df['expected_days']
# compute diffs from dates (preferred)
df['delay_days'] = (df['delivery_date'] - df['expected_date']).dt.days
df['delayed'] = df['delay_days'] > 2
df['cost_per_mile'] = df['cost_usd'] / df['distance_miles']
display(df[['expected_date','delivery_date','delay_days','delayed','cost_per_mile']].head())


,expected_date,delivery_date,delay_days,delayed,cost_per_mile
0,2025-07-27,2025-07-27,0,False,2.700001
1,2025-07-31,2025-07-31,0,False,2.369999
2,2025-07-22,2025-07-25,3,True,3.290001
3,2025-07-02,2025-07-05,3,True,1.850001
4,2025-07-19,2025-07-21,2,False,1.720006


In [15]:
df.to_csv("raw_backup_after_inspect.csv", index=False)

In [16]:
# drop duplicates
df = df.drop_duplicates(subset='shipment_id', keep='first')
# drop essential nulls
df = df.dropna(subset=['shipment_id','delivery_date','expected_date','distance_miles','cost_usd'])
# ensure numeric and positive
df = df[df['distance_miles'] > 0]
# filter implausible cost_per_mile
df = df[df['cost_per_mile'].between(0.2, 20)]
# final quick checks
print("rows after cleaning:", len(df))
display(df.describe().transpose())


rows after cleaning: 500


,count,mean,min,25%,50%,75%,max,std
distance_miles,500.0,1100.82264,110.76,628.725,1104.3,1593.8625,1998.54,539.288321
expected_days,500.0,4.462,2.0,3.0,4.0,6.0,7.0,1.760328
delivery_days,500.0,5.52,1.0,4.0,6.0,7.0,10.0,2.311772
cost_usd,500.0,2770.84308,199.37,1472.1675,2664.705,3835.6725,6708.66,1492.21348
expected_date,500,2025-07-18 01:32:09.600000,2025-07-02 00:00:00,2025-07-10 00:00:00,2025-07-18 00:00:00,2025-07-26 00:00:00,2025-08-04 00:00:00,NaN
delivery_date,500,2025-07-19 02:55:40.800000256,2025-07-05 00:00:00,2025-07-11 00:00:00,2025-07-19 00:00:00,2025-07-27 00:00:00,2025-08-03 00:00:00,NaN
delay_days_by_value,500.0,1.058,-1.0,0.0,1.0,2.0,3.0,1.445966
delay_days,500.0,1.058,-1.0,0.0,1.0,2.0,3.0,1.445966
cost_per_mile,500.0,2.53132,1.5,2.0,2.579999,3.032503,3.490006,0.577038


In [18]:
df.to_csv("clean_shipments.csv", index=False)